In [4]:
#load the necessary packages
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, roc_curve, auc
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

In [5]:
#adapt file paths depending on who's working on them

data_A= "T:/ELS/W_Lisbeth/PHS-CHA/natality22filtered1811_A/natality22filtered1811_A.csv"
data_B= "T:/ELS/W_Lisbeth/PHS-CHA/natality22filtered1811_B/natality22filtered1811_B.csv"
data_C= "T:/ELS/W_Lisbeth/PHS-CHA/natality22filtered1811_C/natality22filtered1811_C.csv"
data_D= "T:/ELS/W_Lisbeth/PHS-CHA/stillbirths2022_filtered.csv"

#load and merge the livebirths datasets

df_A = pd.read_csv(data_A, low_memory=False)
print(df_A.head())
print(df_A.info())

df_B = pd.read_csv(data_B, low_memory=False)
merged_live_df = pd.merge(df_A, df_B, on="seqnum_live", how="outer")

df_C = pd.read_csv(data_C, low_memory=False)
df = pd.merge(merged_live_df, df_C, on="seqnum_live", how="outer")
print(df.head())
print(df.info())


# Merge with stillbirths data
df_D = pd.read_csv(data_D, low_memory=False)
#rename diabetes variable so that it is consistent across datasets
df_D.rename(columns={'rf_diab_num': 'rf_pdiab_num'}, inplace=True)
df_D.rename(columns={'rf_eclam': 'rf_ehype'}, inplace=True)
df_D.rename(columns={'rf_phyp': 'rf_phype'}, inplace=True)
df_D.rename(columns={'rf_ghyp': 'rf_ghype'}, inplace=True)

# Recast common columns
common_columns = set(df.columns) & set(df_D.columns)
exclude_columns = ['rf_ehype', 'rf_ghype', 'rf_phype']
common_columns = [col for col in common_columns if col not in exclude_columns]

# Recast specified columns to float64
for col in common_columns:
    if col in df_D.columns:  # Ensure the column exists
        df[col] = df[col].astype('float64')
        
df_D['stillbirth']=1 #create a flag for the stillbirth rows

#append vertically 
df_D = df_D.reset_index(drop=True)
df = pd.concat([df, df_D], ignore_index=True)
print(len(df))

#recast variables to integers
print(df['livenullipara'].value_counts())
df['livenullipara']=df.livenullipara.replace(np.nan, 9)
print(df['livenullipara'].value_counts())
df['livenullipara']=df['livenullipara'].astype(int)

print(df['livenullipara'].value_counts())
print(df['totnullipara'].value_counts())

df['totnullipara']=df.totnullipara.replace(np.nan, 9)
print(df['totnullipara'].value_counts())

df['totnullipara']=df['totnullipara'].astype(int)
print(df['totnullipara'].value_counts())

print(df['ghyptot'].value_counts())
df['ghyptot']=df.ghyptot.replace(np.nan, 9)

print(df['ghyptot'].value_counts())
df['ghyptot']=df['ghyptot'].astype(int)
print(df['ghyptot'].value_counts())

print(df['interval'].value_counts())
df['interval']=df.interval.replace(np.nan, 9)

print(df['interval'].value_counts())
df['interval']=df['interval'].astype(int)

print(df['interval'].value_counts())
print(df['bmi30'].value_counts())

df['bmi30']=df.bmi30.replace(np.nan, 9)
print(df['bmi30'].value_counts())
df['bmi30']=df['bmi30'].astype(int)
print(df['bmi30'].value_counts())
print(df['priorPTB'].value_counts())
df['priorPTB']=df.priorPTB.replace(np.nan, 9)
print(df['priorPTB'].value_counts())
df['priorPTB']=df['priorPTB'].astype(int)
print(df['priorPTB'].value_counts())
df['priorPTB']=df['priorPTB'].astype(int)
print(df['priorPTB'].value_counts())

print(df['ivf'].value_counts())
df['ivf']=df.ivf.replace(np.nan, 9)
print(df['ivf'].value_counts())
df['ivf']=df['ivf'].astype(int)
print(df['ivf'].value_counts())
print(df['foreignborn'].value_counts())
df['foreignborn']=df.foreignborn.replace(np.nan, 9)

print(df['foreignborn'].value_counts())
df['foreignborn']=df['foreignborn'].astype(int)
print(df['foreignborn'].value_counts())
print(df['rf_phype_num'].value_counts())

print(df['rf_phype_num'].value_counts())
df['rf_phype_num']=df.rf_phype_num.replace(np.nan, 9)
print(df['rf_phype_num'].value_counts())
df['rf_phype_num']=df['rf_phype_num'].astype(int)
print(df['rf_phype_num'].value_counts())

print(df['rf_pdiab_num'].value_counts())
df['rf_pdiab_num']=df.rf_pdiab_num.replace(np.nan, 9)
print(df['rf_pdiab_num'].value_counts())
df['rf_pdiab_num']=df['rf_pdiab_num'].astype(int)
print(df['rf_pdiab_num'].value_counts())

print(df['married'].value_counts())
df['married']=df.married.replace(np.nan, 9)
print(df['married'].value_counts())
df['married']=df['married'].astype(int)
print(df['married'].value_counts())

print(df['nopaternity'].value_counts())
df['nopaternity']=df.nopaternity.replace(np.nan, 9)
print(df['nopaternity'].value_counts())
df['nopaternity']=df['nopaternity'].astype(int)
print(df['nopaternity'].value_counts())

print(df['mother_educ'].value_counts())
df['mother_educ']=df.mother_educ.replace(np.nan, 9)
print(df['mother_educ'].value_counts())
df['mother_educ']=df['mother_educ'].astype(int)
print(df['mother_educ'].value_counts())

print(df['pat_age35'].value_counts())
df['pat_age35']=df.pat_age35.replace(np.nan, 9)
print(df['pat_age35'].value_counts())
df['pat_age35']=df['pat_age35'].astype(int)
print(df['pat_age35'].value_counts())

print(df['father_educ'].value_counts())
df['father_educ']=df.father_educ.replace(np.nan, 9)
print(df['father_educ'].value_counts())
df['father_educ']=df['father_educ'].astype(int)
print(df['father_educ'].value_counts())

print(df['month_pn'].value_counts())
df['month_pn']=df.month_pn.replace(np.nan, 9)
print(df['month_pn'].value_counts())
df['month_pn']=df['month_pn'].astype(int)
print(df['month_pn'].value_counts())

print(df['cig0'].value_counts())
df['cig0']=df.cig0.replace(np.nan, 9)
print(df['cig0'].value_counts())
df['cig0']=df['cig0'].astype(int)
print(df['cig0'].value_counts())

print(df['cig1'].value_counts())
df['cig1']=df.cig1.replace(np.nan, 9)
print(df['cig1'].value_counts())
df['cig1']=df['cig1'].astype(int)
print(df['cig1'].value_counts())

print(df['cig2'].value_counts())
df['cig2']=df.cig2.replace(np.nan, 9)
print(df['cig2'].value_counts())
df['cig2']=df['cig2'].astype(int)
print(df['cig2'].value_counts())

print(df['cig3'].value_counts())
df['cig3']=df.cig3.replace(np.nan, 9)
print(df['cig3'].value_counts())
df['cig3']=df['cig3'].astype(int)
print(df['cig3'].value_counts())

print(df['gwg'].value_counts())
df['gwg']=df.gwg.replace(np.nan, 9)
print(df['gwg'].value_counts())
df['gwg']=df['gwg'].astype(int)
print(df['gwg'].value_counts())

print(df['gdm'].value_counts())
df['gdm']=df.gdm.replace(np.nan, 9)
print(df['gdm'].value_counts())
df['gdm']=df['gdm'].astype(int)
print(df['gdm'].value_counts())

print(df['infection'].value_counts())
df['infection']=df.infection.replace(np.nan, 9)
print(df['infection'].value_counts())
df['infection']=df['infection'].astype(int)
print(df['infection'].value_counts())

print(df['low_income'].value_counts())
df['low_income']=df.low_income.replace(np.nan, 9)
print(df['low_income'].value_counts())
df['low_income']=df['low_income'].astype(int)
print(df['low_income'].value_counts())

print(df['father_race'].value_counts())
df['father_race']=df.father_race.replace(np.nan, 9)
print(df['father_race'].value_counts())
df['father_race']=df['father_race'].astype(int)
print(df['father_race'].value_counts())

print(df['gestation37'].value_counts())
df['gestation37']=df.gestation37.replace(np.nan, 9)
print(df['gestation37'].value_counts())
df['gestation37']=df['gestation37'].astype(int)
print(df['gestation37'].value_counts())

print(df['gestation34'].value_counts())
df['gestation34']=df.gestation34.replace(np.nan, 9)
print(df['gestation34'].value_counts())
df['gestation34']=df['gestation34'].astype(int)
print(df['gestation34'].value_counts())

print(df['priorsb'].value_counts())
df['priorsb']=df.priorsb.replace(np.nan, 9)
print(df['priorsb'].value_counts())
df['priorsb']=df['priorsb'].astype(int)
print(df['priorsb'].value_counts())

print(df['nb_pn'].value_counts())
df['nb_pn']=df.nb_pn.replace(np.nan, 99)
print(df['nb_pn'].value_counts())
df['nb_pn']=df['nb_pn'].astype(int)
print(df['nb_pn'].value_counts())

#additional recoding 
df['nullipar'] = df['livenullipara'].replace(2, 0)
df['int'] = df['interval'].replace(2, 0)
df['setorder_r']=df.setorder_r.replace(np.nan, 9)
ehype_mapping = {'Y': 1, 'N': 0, 'U': 9}
df['rf_ehypenum'] = df['rf_ehype'].map(ehype_mapping)


   dob_mm  mrace6  mhisp_r  frace6  fhisp_r rf_phype rf_ghype rf_ehype  \
0       1       1        0       1        0        N        N        N   
1       1       1        0       1        0        N        N        N   
2       1       1        0       1        0        N        N        N   
3       1       1        0       1        0        N        N        N   
4       1       1        0       1        0        N        Y        N   

   dplural  setorder_r  ...  totnullipara  interval  bmi30  age35  priorPTB  \
0        1           9  ...           1.0       2.0    0.0      1       0.0   
1        1           9  ...           2.0       2.0    0.0      0       0.0   
2        1           9  ...           2.0       2.0    0.0      0       0.0   
3        1           9  ...           2.0       2.0    0.0      1       0.0   
4        1           9  ...           1.0       2.0    0.0      0       0.0   

   ivf  multifetal  foreignborn  foreignres  seqnum_live  
0  0.0           0   

pat_age35
0.0    2178158
1.0    1114301
9.0     403938
Name: count, dtype: int64
pat_age35
0    2178158
1    1114301
9     403938
Name: count, dtype: int64
father_educ
3.0    980849
6.0    670621
4.0    558389
7.0    257217
2.0    251738
5.0    233597
1.0    121414
8.0    112395
9.0     20368
Name: count, dtype: int64
father_educ
3.0    980849
6.0    670621
4.0    558389
9.0    510177
7.0    257217
2.0    251738
5.0    233597
1.0    121414
8.0    112395
Name: count, dtype: int64
father_educ
3    980849
6    670621
4    558389
9    510177
7    257217
2    251738
5    233597
1    121414
8    112395
Name: count, dtype: int64
month_pn
1.0    2778799
2.0     588773
3.0     166766
4.0      79849
9.0       2624
Name: count, dtype: int64
month_pn
1.0    2778799
2.0     588773
3.0     166766
9.0      82210
4.0      79849
Name: count, dtype: int64
month_pn
1    2778799
2     588773
3     166766
9      82210
4      79849
Name: count, dtype: int64
cig0
0.0    3508675
3.0      55544
2.0      55503


# ACOG model estimations

In [6]:
#choose whether to work with 'gyptot' or rf_ehypenum
acog_var=['multifetal','rf_phype_num','rf_pdiab_num','nullipar','bmi30','age35','ivf','africamer'] #excluded ,'priorsb', 'priorPTB','int','low_income'
full_var=['multifetal','rf_phype_num','rf_pdiab_num','nullipar','bmi30','age35','ivf','mother_educ','month_pn','gdm','cig0','cig1','cig2','cig3', 'dob_mm', 'foreignborn', 'foreignres'] #excluded priorsb', 'priorPTB','int','mrace6','married','infection','gwg','low_income','nb_pn',

#exclude twins 
df = df[df['setorder_r'].isin([1, 9])]

# We will work with a subset of the data that only includes rows where relevant variables are not missing
df = df[~df[full_var].isin([9]).any(axis=1)]

#create dataset for acog model
#acog_df = df[acog_var]


## Same Model by using SMOTE. 


Our data for the variable ghyptot is highly imbalanced (over 90% normal patients and less than 1% cases).
We will use SMOTE with random under-sampling to reduce the overall size of the dataset while balancing the classes:

- Random under-sampling to reduce the number of normal cases
- SMOTE to increase the number of anomalies and to equilibrate clases. 

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier


# Store results for each predictor set
final_results = {}

# Function to train, evaluate, and save model results (initial and best models)
def train_and_evaluate_model(model_name, model, param_dist=None):
    print(f"\nTraining {model_name}...")
    
    # Step 1: Train the initial model (before hypertuning)
    model.fit(X_train, y_train)
    y_pred_initial = model.predict(X_test)
    y_prob_initial = model.predict_proba(X_test)[:, 1]
    roc_auc_initial = roc_auc_score(y_test, y_prob_initial)
    clf_report_initial = classification_report(y_test, y_pred_initial, output_dict=True)
    
    # Plot ROC Curve for the initial model
    fpr_initial, tpr_initial, _ = roc_curve(y_test, y_prob_initial)
    plt.figure()
    plt.plot(fpr_initial, tpr_initial, label=f'{model_name} Initial (AUC = {roc_auc_initial:.2f})')
    plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for {model_name} Initial ({predictor_set_name})')
    plt.legend()
    roc_curve_initial_filename = f"roc_curve_{model_name}_initial_{predictor_set_name}.png"
    plt.savefig(roc_curve_initial_filename)
    plt.close()
    
    # Step 2: Hypertune the model (if parameter distribution is provided)
    if param_dist:
        print(f"\nHypertuning {model_name}...")
        search = RandomizedSearchCV(
            estimator=model,
            param_distributions=param_dist,
            n_iter=50,
            scoring='roc_auc',
            cv=3,
            verbose=2,
            random_state=42,
            n_jobs=-1
        )
        search.fit(X_train, y_train)
        best_model = search.best_estimator_
        best_params = search.best_params_
    else:
        best_model = model
        best_params = None
    
    # Step 3: Evaluate the hypertuned model
    y_pred_best = best_model.predict(X_test)
    y_prob_best = best_model.predict_proba(X_test)[:, 1]
    roc_auc_best = roc_auc_score(y_test, y_prob_best)
    clf_report_best = classification_report(y_test, y_pred_best, output_dict=True)
    
    # Plot ROC Curve for the hypertuned model
    fpr_best, tpr_best, _ = roc_curve(y_test, y_prob_best)
    plt.figure()
    plt.plot(fpr_best, tpr_best, label=f'{model_name} Best (AUC = {roc_auc_best:.2f})')
    plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for {model_name} Best ({predictor_set_name})')
    plt.legend()
    roc_curve_best_filename = f"roc_curve_{model_name}_best_{predictor_set_name}.png"
    plt.savefig(roc_curve_best_filename)
    plt.close()
    
    return {
        'Initial Model': {
            'ROC-AUC': roc_auc_initial,
            'Classification Report': clf_report_initial,
            'ROC Curve Image': roc_curve_initial_filename,
            'Best Parameters': None
        },
        'Best Model': {
            'ROC-AUC': roc_auc_best,
            'Classification Report': clf_report_best,
            'ROC Curve Image': roc_curve_best_filename,
            'Best Parameters': best_params
        }
    }

# Loop through predictor sets
for predictor_set_name, predictors in [('All Predictors', full_var), ('ACOG Predictors', acog_var)]:
    print(f"\nTraining models using {predictor_set_name}...")

    # Define X and y
    X = df[predictors]
    y = df['rf_ehypenum']
    
    # Undersample the majority class
    normal_fraction = 0.10
    X_majority = X[y == 0].sample(frac=normal_fraction, random_state=42)
    y_majority = y[y == 0].sample(frac=normal_fraction, random_state=42)
    X_minority = X[y == 1]
    y_minority = y[y == 1]
    
    # Combine reduced majority and full minority
    X_reduced = pd.concat([X_majority, X_minority], axis=0)
    y_reduced = pd.concat([y_majority, y_minority], axis=0)
    
    # Apply SMOTE to balance the classes
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_reduced, y_reduced)
    
    # Split into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X_resampled, y_resampled, test_size=0.3, random_state=42, stratify=y_resampled
    )
    
    # Train Random Forest
    rf_param_dist = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False]
    }
    final_results[(predictor_set_name, 'Random Forest')] = train_and_evaluate_model(
        'Random Forest', RandomForestClassifier(random_state=42), param_dist=rf_param_dist
    )
    
    # Train LightGBM
    lgbm_param_dist = {
        'num_leaves': [31, 50, 70],
        'max_depth': [10, 20, -1],
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [100, 200, 300],
        'min_child_samples': [10, 20, 30]
    }
    final_results[(predictor_set_name, 'LightGBM')] = train_and_evaluate_model(
        'LightGBM', LGBMClassifier(random_state=42), param_dist=lgbm_param_dist
    )
    
    # Train XGBoost
    xgb_param_dist = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 1],
        'colsample_bytree': [0.8, 1],
        'gamma': [0, 1]
    }
    final_results[(predictor_set_name, 'XGBoost')] = train_and_evaluate_model(
        'XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
        param_dist=xgb_param_dist
    )

# Export all results to Excel
flattened_results = []
for (predictor_set_name, model_name), results in final_results.items():
    for model_type, metrics in results.items():
        flattened_results.append({
            'Predictor Set': predictor_set_name,
            'Model': model_name,
            'Model Type': model_type,
            'ROC-AUC': metrics['ROC-AUC'],
            'ROC Curve Image': metrics['ROC Curve Image'],
            'Best Parameters': metrics['Best Parameters']
        })

results_df = pd.DataFrame(flattened_results)
excel_filename = "model_results_rf_lgbm_xgb.xlsx"
with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
    results_df.to_excel(writer, sheet_name="Model Results", index=False)

print(f"Results saved to '{excel_filename}'.")


Training models using All Predictors...

Training Random Forest...

Hypertuning Random Forest...
Fitting 3 folds for each of 50 candidates, totalling 150 fits

Training LightGBM...
[LightGBM] [Info] Number of positive: 257489, number of negative: 257489
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037043 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 876
[LightGBM] [Info] Number of data points in the train set: 514978, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000

Hypertuning LightGBM...
Fitting 3 folds for each of 50 candidates, totalling 150 fits


U:\Users\Waagstein_L.OECDMAIN\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The LGBMClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


[LightGBM] [Info] Number of positive: 257489, number of negative: 257489
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035398 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 876
[LightGBM] [Info] Number of data points in the train set: 514978, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000

Training XGBoost...


U:\Users\Waagstein_L.OECDMAIN\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [20:14:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Hypertuning XGBoost...


U:\Users\Waagstein_L.OECDMAIN\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The XGBClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


AttributeError: 'super' object has no attribute '__sklearn_tags__'

In [7]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

# Define dummy data
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=100, n_features=10, random_state=42)

# Define XGBoost parameter grid
param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'gamma': [0]
}

# Initialize XGBoost classifier
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Initialize RandomizedSearchCV
search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=5,  # Reduce iterations for quick testing
    scoring='roc_auc',
    cv=3,
    random_state=42,
    n_jobs=-1
)

# Run the search
try:
    search.fit(X, y)
    print("RandomizedSearchCV and XGBClassifier are compatible.")
except Exception as e:
    print("Error occurred:", e)


Error occurred: 'super' object has no attribute '__sklearn_tags__'


U:\Users\Waagstein_L.OECDMAIN\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The XGBClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
